In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(url)

In [22]:
import folium

In [2]:
soup = BeautifulSoup(html.text,'html.parser')

In [3]:
# obtain the content of the html
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [27]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [5]:
# loading GeoSpatial Dataset
geo_data=pd.read_csv('Geospatial_Coordinates.csv')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# merge two tables into one
data=df.merge(geo_data,left_on='PostalCode',right_on='Postal Code',how='inner')

In [11]:
data=data.drop(columns='Postal Code')


In [25]:
data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [14]:
!conda install -c conda-forge geopy --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\lingx\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.1               |   py38haa244fe_0         3.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    openssl-1.1.1k             |       h8ffe710_0         5.7 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         8.9 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi            pkgs/main::certifi-202



==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2


openssl-1.1.1k       | 5.7 MB    |            |   0% 
openssl-1.1.1k       | 5.7 MB    |            |   1% 
openssl-1.1.1k       | 5.7 MB    | 2          |   3% 
openssl-1.1.1k       | 5.7 MB    | 6          |   7% 
openssl-1.1.1k       | 5.7 MB    | #2         |  13% 
openssl-1.1.1k       | 5.7 MB    | #7         |  17% 
openssl-1.1.1k       | 5.7 MB    | ##4        |  24% 
openssl-1.1.1k       | 5.7 MB    | ##9        |  30% 
openssl-1.1.1k       | 5.7 MB    | ###7       |  37% 
openssl-1.1.1k       | 5.7 MB    | ####2      |  43% 
openssl-1.1.1k       | 5.7 MB    | ####8      |  48% 
openssl-1.1.1k       | 5.7 MB    | #####3     |  53% 
openssl-1.1.1k       | 5.7 MB    | #####8     |  58% 
openssl-1.1.1k       | 5.7 MB    | ######8    |  69% 
openssl-1.1.1k       | 5.7 MB    | #######5   |  75% 
openssl-1.1.1k       | 5.7 MB    | ########1  |  81% 
openssl-1.1.1k       | 5.7 MB    | ########7  |  87% 
openssl-1.1.1k       | 5.7 MB    | #########2 |  93% 
openssl-1.1.1k       | 5.7 


  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda




In [17]:
from geopy.geocoders import Nominatim

In [20]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [29]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data['Latitude'],data['Longitude'], data['Borough'], data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [30]:
from sklearn.cluster import KMeans

In [31]:
data_cluster=data.drop(columns=['Neighborhood','PostalCode','Borough'])

In [32]:
data_cluster

,Latitude,Longitude
0,43.753259,-79.329656
1,43.725882,-79.315572
2,43.654260,-79.360636
3,43.718518,-79.464763
4,43.662301,-79.389494
...,...,...
98,43.653654,-79.506944
99,43.665860,-79.383160
100,43.662744,-79.321558
101,43.636258,-79.498509


In [33]:
kclusters = 5
kmeans=KMeans(n_clusters=kclusters, random_state=0).fit(data_cluster)
kmeans.labels_[0:10]

array([4, 4, 2, 0, 2, 1, 3, 4, 4, 2])

In [36]:
data.insert(0,'Cluster Labels',kmeans.labels_)

In [37]:
data

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,4,M3A,North York,Parkwoods,43.753259,-79.329656
1,4,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,0,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,2,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...,...
98,1,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,4,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,1,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [39]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data['Latitude'], data['Longitude'], data['Neighborhood'], data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters